In [1]:
import pandas as pd
import re
import os
import json
from slugify import slugify
import pickle

# Processing answers

In [2]:
df = pd.read_json('./raw_answers.jsonl',lines=True)
df.head()

,short_title,linhvuc,cauhoi,traloi,url,askDate,answerDate,answers
0,Thẻ BHYT hết hạn gia hạn lại,Bảo hiểm y tế,\n\nNội dung câu hỏi:\n\n\nKính gửi Quý cơ qua...,\n\nCâu trả lời:\n\n\nQua rà soát dữ liệu mã s...,https://baohiemxahoi.gov.vn/hoidap/pages/defau...,23/02/2023,03/03/2023,"[146/2018/NĐ-CP > điều 12 > khoản 5, 2089/VBHN..."
1,Cách đóng BHYT để không bị gián đoạn sau tốt n...,Sổ BHXH - Thẻ BHYT,\n\nNội dung câu hỏi:\n\n\nEm có câu hỏi như t...,\n\nCâu trả lời:\n\n\n*/ Về đối tượng tham gia...,https://baohiemxahoi.gov.vn/hoidap/pages/defau...,26/02/2023,03/03/2023,"[146/2018/NĐ-CP > điều 5, 146/2018/NĐ-CP > điề..."
2,Khi nào tôi nhận được tiền thai sản,"Ốm đau, thai sản",\n\nNội dung câu hỏi:\n\n\nTôi tên Vũ Thị Tuyế...,"\n\nCâu trả lời:\n\n\nTại Điều 102, Luật Bảo h...",https://baohiemxahoi.gov.vn/hoidap/pages/defau...,22/02/2023,24/02/2023,[58/2014/QH13 > điều 102]
3,LÃNH BHXH 1 LẦN,BHXH tự nguyên,\n\nNội dung câu hỏi:\n\n\nMSBH: 7911379604\nT...,\n\nCâu trả lời:\n\n\n* Về điều kiện hưởng BHX...,https://baohiemxahoi.gov.vn/hoidap/pages/defau...,01/03/2023,10/03/2023,[115/2015/NĐ-CP > điều 8 > khoản 1]
4,Hỗ trợ về tài khoản VssID,Sổ BHXH - Thẻ BHYT,\n\nNội dung câu hỏi:\n\n\nTôi tên là: Trần Th...,\n\nCâu trả lời:\n\n\nQua kiểm tra theo số CCC...,https://baohiemxahoi.gov.vn/hoidap/pages/defau...,22/02/2023,03/03/2023,"[2089/VBHN-BHXH > Điều 27 > khoản 143, 2089/VB..."


In [3]:
REPALCE_DICT = {
    '2089/vbhn-bhxh2089/vbhn-bhxh':'2089/vbhn-bhxh',
    '115/2015/nđ-cp115/2015/nđ-cp':'115/2015/nđ-cp',
    'nghị định số 44/2013/nđ-cp':'44/2013/nđ-cp',
    'quyết định 595/qđ-bhxh':'595/qđ-bhxh',
    'luật an toàn - vệ sinh lao động':'84/2015/qh13'
}
WHITE_LIST_LAW_ID=['58/2014/qh13',
 '2089/vbhn-bhxh',
 '59/2015/tt-blđtbxh',
 '115/2015/nđ-cp',
 '146/2018/nđ-cp',
 '28/2015/nđ-cp',
 '38/2013/qh13',
 '45/2019/qh14',
 '56/2017/tt-byt',
 '134/2015/nđ-cp',
 '28/2021/qđ-ttg',
 '25/2008/qh12',
 '46/2014/qh13',
 '166/qđ-bhxh',
 '61/2020/nđ-cp',
 '33/2016/nđ-cp',
 '505/qđ-bhxh',
 '84/2015/qh13',
 '44/2013/nđ-cp',
 '10/2012/qh13']

In [4]:
def fix(answers):
    new_answers = []
    for i in range(len(answers)):
        answers[i] = answers[i].replace('47.98','47').lower()
        for key in REPALCE_DICT.keys():
            answers[i] = answers[i].replace(key,REPALCE_DICT[key])
        answers[i] = answers[i].split('>')
        answers[i] = [s.strip() for s in answers[i]]
        answers[i] = ' > '.join(answers[i])
        if 'điều' not in answers[i]:
            continue
        for white_key in WHITE_LIST_LAW_ID:
            if answers[i].startswith(white_key+' '):
                new_answers.append(answers[i])
                break
    return new_answers
df['answers'] = df['answers'].apply(fix)

In [5]:
def get_trace(s):
    law_id = None
    dieu = None
    split = s.split(' > ')
    law_id = split[0]
    num = re.compile('điều *(\d+)')
    for i in split:
        result = num.findall(i)
        if len(result)>0:
            dieu=result[0]
    if dieu==None:
        print(s)
    return {
        'law_id':law_id,
        'dieu':dieu
    }
def transform_to_json(answers):
    res = [get_trace(ans) for ans in answers]
    return res


In [6]:
# # get white list law id

# from collections import Counter
# answers = df['answers'].tolist()
# law_ids=[]
# for ans in answers:
#     for row in ans:
#         split = row.split('>')
#         law_id =  split[0].strip()
#         law_ids.append(law_id)
    
# white_list_law_id = []
# for law_id,count in Counter(law_ids).most_common():
#     if count>=5:
#         white_list_law_id.append(law_id)
# white_list_law_id
# Counter(law_ids).most_common()

In [7]:
df['label']=df['answers'].apply(transform_to_json).to_list()

In [8]:
# save
df.to_json('clean_answers.jsonl',orient='records',lines=True,force_ascii=False)

# Processing law document

In [10]:
LAW_ID_MAPPING={
 '58/2014/qh13':'259700',
 '2089/vbhn-bhxh':'454887',
 '59/2015/tt-blđtbxh':'299644',
 '115/2015/nđ-cp':'279974',
 '146/2018/nđ-cp':'357505',
 '28/2015/nđ-cp':'268405',
 '38/2013/qh13':'215628',
 '45/2019/qh14':'333670',
 '56/2017/tt-byt':'341271',
 '134/2015/nđ-cp':'280020',
 '28/2021/qđ-ttg':'489758',
 '25/2008/qh12':'81884',
 '46/2014/qh13':'238506',
 '166/qđ-bhxh':'411816',
 '61/2020/nđ-cp':'393789',
 '33/2016/nđ-cp':'297783',
 '505/qđ-bhxh':'443310',
 '84/2015/qh13':'281961',
 '44/2013/nđ-cp':'187083',
 '10/2012/qh13':'142187'}

In [11]:
# CREATE_CONTENT_TREE=False
# for law_id, tvpl_id in LAW_ID_MAPPING.items():
#     data = get_text(tvpl_id)
#     text= data['content']
#     matches = re.findall(
#                 r'^(Điều (\d+).*)$', text, flags=re.MULTILINE)
#     for i in range(1,len(matches)):
#         if int(matches[i][1])!=int(matches[i-1][1])+1:
#             print('oh shit',law_id,tvpl_id)
#             break
# oh shit 2089/vbhn-bhxh 454887
# oh shit 146/2018/nđ-cp 357505
# oh shit 46/2014/qh13 238506
# oh shit 166/qđ-bhxh 411816 start from 3
# oh shit 61/2020/nđ-cp 393789
# oh shit 505/qđ-bhxh 443310

'58/2014/qh13'